# Importing libraries and reading in the datasets

In [48]:
import pandas as pd
from shapely.wkt import loads
from pyproj import Transformer
import folium
from pykml import parser
from pyproj import proj
import glob

In [49]:
df_newtrees = pd.read_csv('Datasets/newtrees.csv')
df_treescut = pd.read_csv('Datasets/tree_cutting_S-2018.csv', sep=';')
df_Grundwassermessstellen = pd.read_csv('Datasets/Lage_der_Grundwassermessstellen_0.csv', sep=';')
df_Wetterstation_City = pd.read_csv('Datasets/Wetterstation_City.csv')
df_Wetterstation_Uni = pd.read_csv('Datasets/Wetterstation_Universitaet.csv')

# Converting the geodata in df_newtrees 

In [50]:
# Create the ETRS89 and WGS84 projections
etrs89 = proj.CRS.from_epsg(25832)
wgs84 = proj.CRS.from_epsg(4326)

# Create the Transformer to convert ETRS89 coordinates to WGS84 (latitude and longitude)
transformer = proj.Transformer.from_crs(etrs89, wgs84, always_xy=True)

# Extract geolocation data from the "geometry" column and convert to latitude and longitude
def convert_coordinates(geom):
    easting, northing = map(float, geom.strip('POINT ()').split())
    longitude, latitude = transformer.transform(easting, northing)
    return latitude, longitude

df_newtrees['latitude'], df_newtrees['longitude'] = zip(*df_newtrees['_geometry'].apply(convert_coordinates))

In [51]:
df_newtrees

,FID,saison,baumart,link,last_update,_geometry,latitude,longitude
0,stadtbaum.5747bea4-3b38-4b18-8e51-80adf4507d7f,2019/2020,Magnolie,https://www.duesseldorf.de/stadtgruen/baeume-i...,2022-08-09,POINT (345902 5678253),51.234817,6.792643
1,stadtbaum.f008ca3c-82d6-4218-b60f-dce5419f037a,2019/2020,Hainbuche,https://www.duesseldorf.de/stadtgruen/baeume-i...,2022-08-09,POINT (344779 5678180),51.233857,6.776602
2,stadtbaum.4ce92e06-a0dc-4cb0-a0c9-a8baa707277a,2019/2020,Zierapfel,https://www.duesseldorf.de/stadtgruen/baeume-i...,2022-08-09,POINT (345457 5678197),51.234194,6.786298
3,stadtbaum.43e5741c-4a9d-4944-9a32-a624e059dc87,2019/2020,Zierapfel,https://www.duesseldorf.de/stadtgruen/baeume-i...,2022-08-09,POINT (345457 5678232),51.234508,6.786283
4,stadtbaum.0b9841a3-e503-4432-a765-d419cd290b6d,2019/2020,Zierapfel,https://www.duesseldorf.de/stadtgruen/baeume-i...,2022-08-09,POINT (345457 5678257),51.234733,6.786272
...,...,...,...,...,...,...,...,...
384,stadtbaum.1037bb4d-7b0a-45a2-858e-300f6efef64f,2021/2022,Hainbuche,https://www.duesseldorf.de/stadtgruen/baeume-i...,2022-08-09,POINT (353311 5666163),51.128127,6.903630
385,stadtbaum.d3210794-1375-4ce7-9c44-6c69b3eec361,2021/2022,Zierkirsche,https://www.duesseldorf.de/stadtgruen/baeume-i...,2022-08-09,POINT (353632 5666255),51.129036,6.908177
386,stadtbaum.7b1d5cc9-b74b-42d3-aec4-02a791b8d1d0,2021/2022,Zierkirsche,https://www.duesseldorf.de/stadtgruen/baeume-i...,2022-08-09,POINT (353646 5666258),51.129067,6.908376
387,stadtbaum.73457b9d-54f6-4a9e-9e83-a19d7486c084,2021/2022,Kugel-Trompetenbaum,https://www.duesseldorf.de/stadtgruen/baeume-i...,2022-08-09,POINT (353192 5666289),51.129229,6.901879


# Dataset of the trees that were cut recently; Cleaning dataset

In [52]:
df_treescut

,Latitude,Longitude,Altitude,Geometry,Grund der Fällung,Baum_ID,Gattung,Art,Sorte,Deutsch,Straße,Hausnummer,Pflegebezirk,Politischer Bezirk,Stammumfang,X_Koord,Y_Koord
0,51.225187,6.772675,0,point,abgestorben,1032.0,Robinia,pseudoacacia,Monophylla,Robinie,Flinger Straße,li. Von. Nr. 6,3201.0,1.0,NaN,32344475.67,5677224.22
1,51.225252,6.790152,0,point,NaN,1591.0,Robinia,pseudoacacia,Monophylla,Robinie,Hohenzollernstraße,24,NaN,NaN,NaN,32345696.08,5677194.67
2,51.219879,6.778963,0,point,NaN,2190.0,Aesculus,Hippocastanum,NaN,Kastanie,Königsallee,NaN,NaN,NaN,NaN,32344896.81,5676620.84
3,51.215577,6.770085,0,point,NaN,2326.0,Platanus,acerifolia,NaN,Platane,Konkordiastraße,Schule,NaN,NaN,NaN,32344262.40,5676161.23
4,51.226350,6.786974,0,point,NaN,2412.0,Robinia,pseudoacacia,Umbraculifera,Robinie,Liesegangstraße,ggü 11,NaN,NaN,NaN,32345477.85,5677323.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,51.239493,6.803512,0,point,Morschungen im Stammfußbereich,NaN,Populus,*hybrid,NaN,Pappel,Zoopark,NaN,3104.0,2.0,147.0,32346676.20,5678750.10
505,51.239493,6.803512,0,point,Kernfäule im Kronenansatz,NaN,Populus,*hybrid,NaN,Pappel,Zoopark,NaN,3104.0,2.0,334.0,32346676.20,5678750.10
506,51.239493,6.803512,0,point,Morschungen im Stammfußbereich,NaN,Populus,*hybrid,NaN,Pappel,Zoopark,NaN,3104.0,2.0,337.0,32346676.20,5678750.10
507,51.239493,6.803512,0,point,Morschungen im Stammfußbereich,NaN,Populus,*hybrid,NaN,Pappel,Zoopark,NaN,3104.0,2.0,342.0,32346676.20,5678750.10


In [53]:
# In df_treescut: Define a mapping of old reasons for cutting to new values (renaming)
reasons_mapping = {
    'Morschungen im Stammfußbereich': 'Morschungen',
    'Morschungen am Stammfuß': 'Morschungen',  # Example of a different name for the same reason
    'Morschungen am STammbereich': 'Morschungen',
    'Morschung im Stammbereich': 'Morschungen',  
    'Morschungen am Stammbereich': 'Morschungen',
    'Morschungen am Stammfußbereich': 'Morschungen',
    'Morschungen im Stammbereich': 'Morschungen',
    'Morschung': 'Morschungen',
    'Morschungen. Krone ausgebrochen': 'Morschungen',
    'Morschungen am Kronenansatz': 'Morschungen',
    'Morschungen im Kronenansatz': 'Morschungen',
    'Pilzbefall (Lackporling)': 'Disease',
    'Pilzbefall (Lackporling) ': 'Disease',
    'Pilzbefall (Schwefelporling)': 'Disease',
    'Pilzbefall (Schillerporling)': 'Disease',
    'Pilzbefall (Schillerporling) ': 'Disease',
    'Pilzbefall (Riesenporling)': 'Disease',
    'Pilzbefall (Zunderschwamm)': 'Disease',
    'Pilzbefall (Hallimasch)': 'Disease',
    'Pilzbefall (zottiger Schillerporling)': 'Disease',
    'Pilzbefall (schuppiger Porling)': 'Disease',
    'Pilzbefall (sparriger Schüppling)': 'Disease',
    'Pilzbefall am Wurzelhals (Schillerporling)': 'Disease',
    'Pilzbefall am Wurzelhals (Lackporling)': 'Disease',
    'Pilzbefall am Wurzelhals': 'Disease',
    'Pilzbefall': 'Disease',
    'Komplexerkrankung': 'Disease',
    'Rußrindenkrankheit': 'Disease',
    'Riss im Zwiesel': 'Riss im Zwieselbereich',
    'Sturmschaden ': 'Sturmschaden',
    'Hochspannungsleitung Spannungsüberschlag': 'Unfallschaden',
    'Wurzelfäule': 'Wurzelschäden',
    'Neubaumaßnahme ': 'Baumaßnahme',
    'Zwiesel am Stammfuß': 'Stammschäden',
    'Riss im Zwieselbereich': 'Stammschäden',
    'Hohlstellen am Stammfuß': 'Stammschäden',
    'Vandalismus Verursacher unbekannt': 'Vandalismus',
    'nur noch Stamm': 'Torso',
    'Sturmschaden': 'Sturm-, Gebäude- und Unfallschäden',
    'Gebäudeschäden': 'Sturm-, Gebäude- und Unfallschäden',
    'Unfallschaden': 'Sturm-, Gebäude- und Unfallschäden',
    'Sturm- und Gebäudeschäden': 'Sturm-, Gebäude- und Unfallschäden',
    'abgestorben': 'dead',
    'Sturm-, Gebäude- und Unfallschäden': 'Storm, building and accident damage',
    'Stammschäden': 'Trunk damage',
    'Riss im Zwieselbereich':'Crack in the Zwiesel area',
    'Wurzelschäden': 'Root damage',
    'Unfallgefahr': 'Risk of accident',
    'Vandalismus': 'Vandalism',
    'Unfallschaden': 'Storm, building and accident damage',
    'Baumaßnahme': 'Construction measure',
    'Sturmschaden': 'Storm, building and accident damage',
    'Kernfäule im Kronenansatz': 'Core rot in the crown base'
}

# Replace the values in the "Grund der Fällung" column using the mapping
df_treescut['Grund der Fällung'] = df_treescut['Grund der Fällung'].replace(reasons_mapping)

# Get the unique values in the "Grund der Fällung" column after renaming
unique_reasons = df_treescut['Grund der Fällung'].unique()

# Print the unique reasons for cutting after renaming
print(unique_reasons)


['dead' nan 'Torso' 'Morschungen' 'Storm, building and accident damage'
 'Disease' 'Stammschäden' 'Trunk damage' 'Riss im Zwieselbereich'
 'Wurzelschäden' 'Sturm-, Gebäude- und Unfallschäden' 'Risk of accident'
 'Crack in the Zwiesel area' 'Vandalismus' 'Unfallschaden'
 'Construction measure' 'Sturmschaden' 'Baumaßnahme' 'Root damage'
 'Core rot in the crown base']


In [54]:
#Creating a map object and setting the initial view with the latitude and longitude of Düsseldorf
duesseldorf_coords = (51.2277, 6.7735)
map_tree_location = folium.Map(location=duesseldorf_coords, tiles="cartodb positron", zoom_start=12)

#Creating a second map object and setting the initial view with the latitude and longitude of Düsseldorf
map_tree_location2 = folium.Map(location=duesseldorf_coords, tiles="cartodb positron", zoom_start=12)

In [55]:
#Iterating through df_newtrees containing the tree locations and tree type, and add markers for each tree location to the map
for index, row in df_newtrees.iterrows():
    tree_coords1 = (row['latitude'], row['longitude'])
    popup_content1 = f"Tree Type: {row['baumart']}, Planting Year: {row['saison']}"
    marker1 = folium.Marker(location=tree_coords1, popup=popup_content1, icon=folium.Icon(color='green', icon='leaf')) 
    marker1.add_to(map_tree_location)

In [56]:
#Iterating through df_newtrees containing the tree locations and tree type, and add markers for each tree location to the second map
for index, row in df_newtrees.iterrows():
    tree_coords1 = (row['latitude'], row['longitude'])
    popup_content1 = f"Tree Type: {row['baumart']}, Planting Year: {row['saison']}"
    marker1 = folium.Marker(location=tree_coords1, popup=popup_content1, icon=folium.Icon(color='green', icon='leaf')) 
    marker1.add_to(map_tree_location2)
    
    #Iterating through df_treescut containing the tree locations and tree type, and add markers for each tree location to the second map
for index, row in df_treescut.iterrows():
    tree_coords2 = (row['Latitude'], row['Longitude'])
    popup_content2 = f"Tree Type: {row['Deutsch']}, Reason for cutting: {row['Grund der Fällung']}"
    marker2 = folium.Marker(location=tree_coords2, popup=popup_content2,  icon=folium.Icon(color='red', icon='leaf')) 
    marker2.add_to(map_tree_location2) 
    

In [57]:
#Save the map as an HTML file or display it directly in a Jupyter Notebook
map_tree_location.save('new_trees_map.html')

#Save the second map as an HTML file or display it directly in a Jupyter Notebook
map_tree_location2.save('new_and_cutted_trees_map.html')

In [58]:
map_tree_location

In [59]:
map_tree_location2

# Weather Data (Adding the coordinates)

In [60]:
# dropping the column "Unnamed: 0" in the weather datasets
df_Wetterstation_City = df_Wetterstation_City.drop(columns = "Unnamed: 0") 
df_Wetterstation_Uni = df_Wetterstation_Uni.drop(columns = "Unnamed: 0") 

In [61]:
### Adding the coordinates for the two weather stations

# Adding a new column "Latitude" to the two weather datasets
df_Wetterstation_City["Latitude"] = 51.24
df_Wetterstation_Uni["Latitude"] = 51.19

# Adding a new column "Longitude" to the two weather datasets
df_Wetterstation_City["Longitude"] = 6.79
df_Wetterstation_Uni["Longitude"] = 6.80

In [62]:
df_Wetterstation_City.head(100)

,Datum,Tmit,Sges,Rges,Latitude,Longitude
0,01.01.2012,"12,1",0,"5,7",51.24,6.79
1,02.01.2012,"8,3",0,"3,0",51.24,6.79
2,03.01.2012,"8,2",0,"2,8",51.24,6.79
3,04.01.2012,"6,8","0,7","5,7",51.24,6.79
4,05.01.2012,7,"0,5","17,4",51.24,6.79
...,...,...,...,...,...,...
95,05.04.2012,"7,1",0,"0,0",51.24,6.79
96,06.04.2012,"7,7","6,1","2,9",51.24,6.79
97,07.04.2012,"5,6","1,4","0,1",51.24,6.79
98,08.04.2012,"6,4","10,2","9,2",51.24,6.79


In [63]:
df_Wetterstation_Uni

,Datum,Tmit,Sges,Rges,Latitude,Longitude
0,01.04.2012,"5,2","10,4","0,0",51.19,6.8
1,02.04.2012,"6,4","2,6","0,0",51.19,6.8
2,03.04.2012,10,"6,7","0,5",51.19,6.8
3,04.04.2012,"9,2","1,4","0,0",51.19,6.8
4,05.04.2012,"6,8",0,"0,0",51.19,6.8
...,...,...,...,...,...,...
3160,27.12.2020,"4,6",0,"6,5",51.19,6.8
3161,28.12.2020,"4,1","2,3","0,6",51.19,6.8
3162,29.12.2020,"1,7","0,1",0,51.19,6.8
3163,30.12.2020,"3,2",0,"2,8",51.19,6.8


# Combining the datasets df_Grundwassermessstellen and the csv files with the groundwater levels into one dataset: df_groundwater

In [101]:
# Define the pattern for the CSV files (e.g., all CSV files in the current directory)
csv_files_pattern = 'Datasets/Groundwater/*.csv'  # Update the path to your dataset directory

# Get a list of file paths that match the pattern using glob
csv_file_paths = glob.glob(csv_files_pattern)

# Create an empty list to store DataFrames after processing
processed_dataframes = []

# Loop through each file path
for file_path in csv_file_paths:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path, sep=';', encoding="cp1252")
    
    # Get the values from the first row for 'Latitude', 'Longitude', and 'Street'
    latitude_value = df.loc[0, 'Latitude']
    longitude_value = df.loc[0, 'Longitude']
    street_value = df.loc[0, 'Street']
    
    # Fill the corresponding columns with the extracted values
    df['Latitude'] = latitude_value
    df['Longitude'] = longitude_value
    df['Street'] = street_value
    
    # Append the processed DataFrame to the list
    processed_dataframes.append(df)

# Concatenate the processed DataFrames
df_groundwater = pd.concat(processed_dataframes, ignore_index=True)

# Dropping the column 'Unnamed: 5' (Don't know where that came from)
df_groundwater = df_groundwater.drop(columns=['Unnamed: 5'])

# Renaming the columns x, y
df_groundwater.rename(columns = {'x':'Date', 'y':'monthly_groundwater'}, inplace = True)

In [102]:
df_groundwater 

,Date,monthly_groundwater,Latitude,Longitude,Street
0,29.06.1972,"38,21",51.206.063,6.860.008,Am Ellerforst
1,25.07.1972,"38,41",51.206.063,6.860.008,Am Ellerforst
2,15.08.1972,"38,33",51.206.063,6.860.008,Am Ellerforst
3,20.09.1972,"38,19",51.206.063,6.860.008,Am Ellerforst
4,11.10.1972,"38,06",51.206.063,6.860.008,Am Ellerforst
...,...,...,...,...,...
230910,29.07.2023,"25,95",51.227.636,6.758.231,Wildenbruchstraße
230911,30.07.2023,"26,08",51.227.636,6.758.231,Wildenbruchstraße
230912,31.07.2023,"26,21",51.227.636,6.758.231,Wildenbruchstraße
230913,01.08.2023,"26,31",51.227.636,6.758.231,Wildenbruchstraße
